In [2]:
from datasets import load_dataset
from tensorflow.keras.layers import Dense, GRU, Embedding
import sentencepiece as spm
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
import spacy
from tqdm import tqdm
from collections import Counter
import pandas as pd
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
!pip install fr_core_news_sm

2025-10-15 07:44:13.617164: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760514253.793614      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760514253.854895      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 94.8 MB/s eta 0:00:00:00:0100:01


In [3]:
dataset = load_dataset("opus_books", "en-fr")
print(dataset['train'][0])

README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

{'id': '0', 'translation': {'en': 'The Wanderer', 'fr': 'Le grand Meaulnes'}}


In [4]:
import pandas as pd
df = pd.DataFrame(dataset['train'])

df['english'] = df['translation'].apply(lambda x: x['en'])
df['french'] = df['translation'].apply(lambda x: x['fr'])

df2 = df[['english', 'french']]

In [5]:
df.head()

,id,translation,english,french
0,0,"{'en': 'The Wanderer', 'fr': 'Le grand Meaulnes'}",The Wanderer,Le grand Meaulnes
1,1,"{'en': 'Alain-Fournier', 'fr': 'Alain-Fournier'}",Alain-Fournier,Alain-Fournier
2,2,"{'en': 'First Part', 'fr': 'PREMIÈRE PARTIE'}",First Part,PREMIÈRE PARTIE
3,3,"{'en': 'I', 'fr': 'CHAPITRE PREMIER'}",I,CHAPITRE PREMIER
4,4,"{'en': 'THE BOARDER', 'fr': 'LE PENSIONNAIRE'}",THE BOARDER,LE PENSIONNAIRE


In [6]:
# ---------------
# Lower casing words
# ---------------

nlp_en = spacy.load('en_core_web_sm', disable = ["parser"]) # "disable = 'parser'" means disabling parsing. Parsing creates unnecessary tree-based relations for each word to identify whether that particular word is an adjective, noun or pronoun. So, it not necessary at all and disabling it fastens the process.
nlp_fr = spacy.load('fr_core_news_sm', disable = ["parser"]) # This is for french grammar.

def smart_case(sentences, nlp, batch_size = 1000): # This is an advanced smart_case which is built in order to smartly lower-case the required sentence using batches because, the lower casing may take a lot of time. Also, this will take all the sentences at once.
    
    processed = []
    
    # nlp.pipe automatically batches sentences → much faster
    for doc in tqdm(nlp.pipe(sentences, batch_size = batch_size, n_process = -1), total = len(sentences)): # 'n_process = -1' will use all the CPU cores for processing this. 'tqdm' will show real time processing updates. The 'doc' is a the entire single sentence.
        tokens = [] # Here, separate list of tokens will be created for each sentence.
        for token in doc: # 'token' captures each word of that sentence.
            if token.ent_type_ or token.pos_ in ['PROPN'] or token.text.isupper():
                tokens.append(token.text)

            else:
                tokens.append(token.text.lower())

        processed.append(' '.join(tokens)) 

    return processed
        
english_sentences = smart_case(df['english'].tolist(), nlp_en)
french_sentences = smart_case(df['french'].tolist(), nlp_fr)



100%|██████████| 127085/127085 [05:07<00:00, 413.07it/s]


In [4]:
import json

# with open("english_sentences.json", 'w', encoding = 'utf-8') as f:
#     json.dump(english_sentences, f, ensure_ascii = False, indent = 2) # ensure_ascii  = False will prevent to treat non-ascii values to something else - saving the text as it is.

# with open("french_sentences.json", 'w', encoding = 'utf-8') as f:
#     json.dump(french_sentences, f, ensure_ascii = False, indent = 2)


with open("/kaggle/input/english-french-lower-cased/english_sentences.json", 'r', encoding = 'utf-8') as f:
    english_sentences = json.load(f)

with open("/kaggle/input/english-french-lower-cased/french_sentences.json", 'r', encoding = 'utf-8') as f:
    french_sentences = json.load(f)

In [5]:
print(english_sentences[:5])
print(french_sentences[:5])

['the wanderer', 'Alain - Fournier', 'First Part', 'I', 'THE BOARDER']
['le grand Meaulnes', 'Alain - Fournier', 'PREMIÈRE PARTIE', 'CHAPITRE PREMIER', 'LE PENSIONNAIRE']


In [6]:
french_sentences_input = ['<start> ' + s for s in french_sentences] # This creates a pattern for model saying that after '<start>' the translation should begin and after '<end>' translation should stop for every sentence in decoders.
french_sentences_output = [s + ' <end>' for s in french_sentences]
english_sentences_input = [s for s in english_sentences]

print(french_sentences_input[:5])
print(french_sentences_output[:5])

['<start> le grand Meaulnes', '<start> Alain - Fournier', '<start> PREMIÈRE PARTIE', '<start> CHAPITRE PREMIER', '<start> LE PENSIONNAIRE']
['le grand Meaulnes <end>', 'Alain - Fournier <end>', 'PREMIÈRE PARTIE <end>', 'CHAPITRE PREMIER <end>', 'LE PENSIONNAIRE <end>']


In [ ]:
# -------Checking important symbols in the English sentences by finding all non-alphabet characters-------
import re

all_text = ' '.join(english_sentences_input) 


special_characters = re.findall('[^a-zA-Z]', all_text)

# This will count frequency of each symbol
freq_counter = Counter(special_characters)

print(freq_counter)


Counter({' ': 3256727, ',': 223733, '.': 124433, '"': 72898, "'": 29725, ';': 27424, '-': 26275, 'é': 19581, '’': 17346, '!': 15760, '?': 13344, ':': 7508, 'à': 4937, '−': 2686, 'è': 2591, '_': 2043, 'ê': 1990, '0': 1847, '1': 1532, '—': 1408, '–': 1110, 'â': 887, 'ç': 852, '(': 839, ')': 839, '2': 836, '«': 801, 'ô': 745, '»': 652, '5': 581, '3': 571, 'û': 545, 'É': 543, 'î': 530, '~': 512, '8': 481, '6': 476, '4': 459, '7': 396, '“': 367, '”': 366, '*': 364, 'ù': 359, 'œ': 320, '…': 300, '9': 253, '#': 142, 'æ': 68, 'ï': 67, '°': 56, '&': 54, 'ä': 45, '[': 34, '‘': 33, ']': 32, '`': 24, 'ö': 16, 'ë': 13, '$': 10, '>': 8, 'α': 8, '£': 7, 'ο': 7, 'ü': 5, 'γ': 5, 'ν': 5, 'τ': 5, '/': 4, 'ã': 4, 'Ç': 3, 'А': 3, 'Г': 3, 'Н': 3, 'ί': 3, 'κ': 3, '%': 3, 'Æ': 3, 'έ': 2, 'β': 2, '=': 2, '{': 1, '}': 1, 'ι': 1, 'λ': 1, 'μ': 1, 'ό': 1, 'φ': 1, 'а': 1, 'ε': 1, 'ϊ': 1, 'Œ': 1, '+': 1})


In [ ]:
# ------ Finding Max Length based on the 90 percentile ranking -------

eng_lengths = [len(s) for s in english_sentences_input]
eng_max_len = int(np.percentile(eng_lengths, 90))
print(eng_max_len)

fr_lengths = [len(s) for s in french_sentences_input]
fr_max_len = int(np.percentile(fr_lengths, 90))
print(fr_max_len)

238
254


In [ ]:
# ------------------
# Subword Tokenization
# ------------------
# Instead of Tokenization which leads to OOM problem, I am using subword tokenization which reduces the vocabulary size. E.g., if 'Playful' is a rare word and 'Play' is a common word then, the sentencepiece will convert 'Playful' word into sub words: 'Play' & 'ful', now both the words share a common vocabulary 'Play'. Since we know that in our eng_vocab we have 70k words and there can be many rare words in the vocabulary, so, if we do the subword tokenization for those thousands of words which are rare then, this may decrease the a large amount of vocab size which further prevents OOM problem as this prevents the creation units for uncommon-vocabularies in dense layer, as parameter in dense usually increases if no. of units or vocab_size is larges as parameter_size in Dense = units x vocab_size.

with open("eng.txt", "w", encoding="utf-8") as f:
    for s in english_sentences:
        f.write(s + "\n") # This will make the entire sentences joined with but with '\n'(because, sentence piece wants each sentence to be like that), which will join all the sentences into a single string.

with open("fr.txt", "w", encoding="utf-8") as f:
    for s in french_sentences_input + french_sentences_output:
        f.write(s + "\n")
        
spm.SentencePieceTrainer.Train(input = 'eng.txt',# 'input' will save each sentence for training.
                               model_prefix = 'eng_sp', vocab_size = 8000, model_type = 'bpe') # 'model_prefix = 'eng_sp'('english_sentencepiece') is the name of which will be given to our eng_sen after training. 'model_type = bpe' will use the byte pair encoding(e.g., 'Playing' becomes 'Play', 'ing' on the basis of frequency). vocab_size = 8000-12000 is mostly used for sentences in range(120k-500k)

spm.SentencePieceTrainer.Train(input = 'fr.txt',
                              model_prefix = 'fr_sp', vocab_size = 8000, model_type = 'bpe' )


sp_eng = spm.SentencePieceProcessor()
sp_eng.Load("/kaggle/input/sentence-piece-models/eng_sp.model")

sp_fr = spm.SentencePieceProcessor()
sp_fr.Load("/kaggle/input/sentence-piece-models/fr_sp.model")

eng_seq = [sp_eng.EncodeAsIds(s) for s in english_sentences_input]
fr_seq_in = [sp_fr.EncodeAsIds(s) for s in french_sentences_input]
fr_seq_out = [sp_fr.EncodeAsIds(s) for s in french_sentences_output]

eng_seq = pad_sequences(eng_seq, maxlen = eng_max_len, padding = 'post') # Given 'maxlen' to avoid overfitting


fr_seq_in = pad_sequences(fr_seq_in, maxlen = fr_max_len, padding = 'post')
fr_seq_out = pad_sequences(fr_seq_out, maxlen = fr_max_len, padding = 'post')


# Vocabulary size
eng_vocab = (sp_eng.GetPieceSize()) + 1
fr_vocab = (sp_fr.GetPieceSize()) + 1
eng_input_len = eng_seq.shape[1]
fr_input_len = fr_seq_in.shape[1]


# ---------------------------
# Building Encoder-Decoder model
# --------------------------
embedding_dim = 128 # change later if it underfits
units =  256 # 256-338 is enough for 8k vocab size
drop = 0.3

# Encoder
encoder_inputs = Input(shape = (eng_input_len, ))  # This input is written has a shape of (max_eng_len, ) instead of (1, max_eng_len) because, doing (1, max_eng_len)  will make each word as a feature of a sentence which the model doesn't expect instead, since we know that model train all the sentences on the basis of batch size, so, we do (max_eng_len, ) in order to state that "Hey model take this input_len which is the maximum input len of all the english sentence. Later, you'll add a batch size to it in order to train sentence at each time_step".
encoder_emb = Embedding(eng_vocab,embedding_dim, mask_zero = True)(encoder_inputs)
encoder_emb = Dropout(drop)(encoder_emb)
encoder_lstm = LSTM(units, return_state = True, dropout = drop, recurrent_dropout = drop)
_, state_h, state_c = encoder_lstm(encoder_emb)
encoder_state = [state_h, state_c] # Both states needed


# Decoder
decoder_inputs = Input(shape = (fr_input_len, ))
decoder_emb = Embedding(fr_vocab, embedding_dim, mask_zero = True)(decoder_inputs) # 'mask_zero = True' tells the keras that ignore the timesteps which are '0'. Embeddding will not happen to those timesteps which have '0' values as well as model will ignore them due to paddings. Always give this parameter.
decoder_emb = Dropout(drop)(decoder_emb)
decoder_lstm = LSTM(units, return_sequences = True, return_state = True, dropout = drop, recurrent_dropout = drop)
decoder_outputs, _, _ = decoder_lstm(decoder_emb, initial_state = encoder_state)
decoder_dense = Dense(fr_vocab, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs) # Capturing the final output of the decoder. The ouput returns the probability vector for each timestep. Shape = (batch_size, timestep, units)


# Seq2Seq Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs) # Now, creating a model that will act as a pipeline which takes both 'Encdoer' and 'Decoder' inputs-outpus and convert encoder-decoder into a single trainable unit.
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()


# -------------
# Training Model
#--------------
model.fit([eng_seq, fr_seq_in], np.expand_dims(fr_seq_out, -1), # This is called teacher forcing — we teach the model about what the correct “previous words” were at each timestep, and that is the reason why we are giving 'fr_in_seq' along with 'en_seq'. We did ' np.expand_dims(fr_out_seq, -1)' because as we know that (n, -1) will make a 2D vector('2D vector' because, earlier 'fr_out_seq' was in the shape of (n, ) which is a list like shape and which is a 1D vector). sparse_categorical_crossentropy needs an extra feature dimension so provide it by using '-1' which means - fr_out_seq shape before expand_dims: (num_samples, max_fr_len) → 2D, After np.expand_dims(..., -1): (num_samples, max_fr_len, 1) → 3D. Now Keras can match it with the model output shape (num_samples, max_fr_len, vocab_fr).
            batch_size = 16, epochs = 200, verbose = 2, validation_split = 0.1, callbacks = [EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)])
print("Training done!")


# ------------------------
# Building Inference Models: # This Inference phase → uses the learned patterns to generate predictions one token at a time. Encoder inference model — gives us the encoder state for the input sentence. Decoder inference model — takes one word at a time and predicts the next word. The Inference phase is a testing phase.
# -----------------------
# Encoder Inference Model
encoder_model = Model(encoder_inputs, encoder_state) # It takes encoder inputs which is the input(test) sentence and the 'encoder_state' as an output context, and later, this context vector will be predicted by the model during translation.


# Decoder Inference Model
decoder_state_input_h = Input(shape = (units, )) # This creates a placeholder for the decoder’s previous hidden state(you just try to understand the below decoder_model's prediction working in order to understand it). Remember: during translation, at each loop iteration, we’ll feed the decoder the last predicted hidden state. So this decoder_state_input acts like: “Hey decoder, here’s where you were in the previous word.”
decoder_state_input_c = Input(shape = (units, )) 
dec_emb2 = Embedding(fr_vocab, embedding_dim,  mask_zero = True)(decoder_inputs) # We are sending 'decoder_inputs' as input shape even though later on, the input will be a single word because, in the 'Embedding', the size of the input is not rigid to the specified size to the 'Embedding'(like decoder_inputs). Also, the 'Embedding' layer only wants a condition to be true while handling that single input is that the input must be having the specified embedding dimensions(like embedding_dim which we already gave, so everything is fine).
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = [decoder_state_input_h, decoder_state_input_c])  # 'initial_state=[decoder_state_input_h, decoder_state_input_c])' → this tells LSTM: “Start processing this word from where you left off last time.”
decoder_outputs2 = decoder_dense(decoder_outputs2) # This is the final Dense layer (softmax over vocabulary). It converts the GRU output (hidden vector) into vocabulary probabilities.So if vocab_fr = 5000, then this gives a vector like: [0.01, 0.05, 0.70, 0.02, ...] meaning --> “70% chance the next word is token 3.”
decoder_model = Model([decoder_inputs, decoder_state_input_h, decoder_state_input_c], # Inputs: [decoder_inputs, decoder_state_input_h, decoder_state_input_c] → The model will take both the current token and the previous hidden and cell states. 
                     [decoder_outputs2, state_h2, state_c2]) # Outputs: [decoder_outputs2, state_h2, state_c2] → It will return the predicted next-token probabilities and the updated hidden and cell states.


# -----------------------
# Translation(Prediction): # The working of this is written in the markdown notes.
# -----------------------
reverse_fr_index = {i: w for w, i in fr_tokenizer.word_index.items()} # This will form a dictionary - used later.

def translate_sentence(sentence):
    sentence = ' '.join(smart_case([sentence], nlp_en))
    seq = eng_spm.EncodeAsIds(sentence)
    seq = pad_sequences([seq], maxlen = eng_max_len, padding = 'post') # Since we pass lists of tokenized sentences to pad_sequences(see the above eng_seq output).
    state = encoder_model.predict(seq) # Because encoder_model returns two states

    decoded_sentence = []
    start_id = fr_spm.PieceToId('<start>')
    input_id = np.array([[start_id]])
    
    stop = False

    while not stop:
        output_, output_h, output_c = decoder_model.predict([input_id] + state)
        output_id = np.argmax(output_[0, -1, :])
        word = fr_spm.DecodeIds([output_id])

        if word == '<end>' or len(decoded_sentence) > fr_max_len:
            stop = True

        else:
            decoded_sentence.append(word)
            input_id = np.array([[output_id]])
            state = [output_h, output_c]

    return ' '.join(decoded_sentence)

In [ ]:
# ----------
# Testing-Phase
# ----------
sentences = ["I love eating apples.", "Please forgive me.", "Stay alert!"]
predictions = []

for s in sentences:
    predictions.append(sentence_translation(s))
    print(s)


# ----------------------
# Evaluation-Phase
# ----------------------
from nltk.translate.bleu_score import sentence_bleu
scores = []
references = ["J'adore manger des pommes.","Pardonnez-moi, s'il vous plaît.", "Restez vigilant(e) !"]

for ref, pred in zip(references, predictions):
    score = sentence_bleu([ref.split()], [pred.split()])
    scores.append(score)

print(f"Average Bleu Score: {sum(scores) / len(scores)}.:4")

# </br> Using Bidirectional LSTM without attention. Note, model is not trained because of large sequences and when I drop some sequences then the accuracy decreases which we do not want.

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, Input, Embedding, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import sentencepiece as spm
from datasets import load_dataset
import spacy
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences
!pip install fr_core_news_sm

2025-10-16 06:21:23.201228: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760595683.394976      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760595683.450514      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 38.6 MB/s eta 0:00:0000:0100:01


In [2]:
data = load_dataset("opus_books", "en-fr")
print(data)

README.md: 0.00B [00:00, ?B/s]

en-fr/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})


In [3]:
df = pd.DataFrame(data['train'])
df['english'] = df['translation'].apply(lambda x: x['en'])
df['french'] = df['translation'].apply(lambda x: x['fr'])

df2 = df[['english', 'french']]

df3 = df2.head(30000)


In [10]:
nlp_en = spacy.load('en_core_web_sm', disable = ['parser'])
nlp_fr = spacy.load('fr_core_news_sm', disable = ['parser'])

def smart_case(sentences, nlp, batch_size = 1000):
    processed = []
    
    for doc in tqdm(nlp.pipe(sentences, batch_size = batch_size, n_process = -1), total = len(sentences)):
        tokens = []

        for token in doc:
            
            
            if token.ent_type_ or token.pos_ in ['PROPN'] or token.text.isupper():
                tokens.append(token.text)

            else:
                tokens.append(token.text.lower())

        processed.append(' '.join(tokens))
        
    return processed

english_sentences = smart_case(df3['english'].tolist(), nlp_en)
french_sentences = smart_case(df3['french'].tolist(), nlp_fr)


100%|██████████| 30000/30000 [01:27<00:00, 341.61it/s]


In [5]:
import json

# with open("english_smartCased_sentences.json", "w", encoding = 'utf-8') as f:
#     json.dump(english_sentences, f, ensure_ascii = False, indent = 2)

# with open("french_smartCased_sentences.json", "w", encoding = 'utf-8') as f:
#     json.dump(french_sentences, f, ensure_ascii = False, indent = 2)

with open("/kaggle/input/smart-cased-sentences/english_smartCased_sentences.json", "r", encoding = 'utf-8') as f:
    english_sentences = json.load(f)

with open("/kaggle/input/smart-cased-sentences/french_smartCased_sentences.json", "r", encoding = 'utf-8') as f:
    french_sentences = json.load(f)



In [ ]:
print(english_senteces[:5])
print(french_sentences[:5])

In [8]:
french_input_sentences = ['<start>' + s for s in french_sentences]
french_output_sentences = [s + '<end>' for s in french_sentences]



In [ ]:
with open("en.txt", "w", encoding = 'utf-8') as f:
    for s in english_sentences:
        f.write(s + "\n")

with open("fr.txt", "w", encoding = 'utf-8') as f:
    for s in french_sentences:
        f.write(s + "\n")

spm.SentencePieceTrainer.Train(input = "en.txt", vocab_size = 3000, model_type = "bpe", model_prefix = "eng_sp")
spm.SentencePieceTrainer.Train(input = "fr.txt", vocab_size = 3000, model_type = "bpe", model_prefix = "fr_sp")

In [12]:
eng_spm = spm.SentencePieceProcessor()
eng_spm.load("eng_sp.model")

fr_spm = spm.SentencePieceProcessor()
fr_spm.load("fr_sp.model")

eng_seq = [eng_spm.EncodeAsIds(s) for s in english_sentences]
fr_seq_in = [fr_spm.EncodeAsIds(s) for s in french_input_sentences]
fr_seq_out = [fr_spm.EncodeAsIds(s) for s in french_output_sentences]



In [15]:
lengths = [len(s) for s in english_sentences]
eng_max_len = int(np.percentile(lengths, 90))

lengths2 = [len(s) for s in french_sentences]
fr_max_len = int(np.percentile(lengths2, 90))

eng_vocab = eng_spm.GetPieceSize() + 1
fr_vocab = fr_spm.GetPieceSize() + 1

In [ ]:
embedding_dim = 64
units = 128
dropout = 0.2

eng_seq = pad_sequences(eng_seq, padding = 'post', maxlen = eng_max_len)
fr_seq_in = pad_sequences(fr_seq_in, padding = 'post', maxlen = fr_max_len)
fr_seq_out = pad_sequences(fr_seq_out, padding = 'post', maxlen = fr_max_len)

# Encoder
encoder_input = Input(shape = (eng_max_len,))
encoder_emb = Embedding(eng_vocab, embedding_dim, mask_zero = True)(encoder_input)
encoder_emb = Dropout(dropout)(encoder_emb)
encoder_biLstm = Bidirectional(LSTM(units, return_state = True, dropout = dropout, recurrent_dropout = dropout))
encoder_output, forward_h, forward_c, backward_h, backward_c = encoder_biLstm(encoder_emb)
combined_h = Concatenate()([forward_h, backward_h])
combined_c = Concatenate()([forward_c, backward_c])

# Decoder
decoder_input = Input(shape = (fr_max_len, ))
decoder_emb = Embedding(fr_vocab, embedding_dim, mask_zero = True)(decoder_input)
decoder_emb = Dropout(dropout)(decoder_emb)

decoder_lstm = LSTM(units*2, return_sequences = True, return_state = True, dropout = dropout)
decoder_output, _, _ = decoder_lstm(decoder_emb, initial_state = [combined_h, combined_c])
decoder_dense = Dense(fr_vocab, activation = "softmax")
decoder_output = decoder_dense(decoder_output)


# Model Training
model = Model([encoder_input, decoder_input],decoder_output)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit([eng_seq, fr_seq_in], fr_seq_out, validation_split = 0.1, epochs = 200, batch_size = 512, callbacks = [EarlyStopping(monitor = 'val_loss', patience = 8, restore_best_weights = True)])
print(model.summary())

# Seq2Seq Model
# Encoder Inference Model
encoder_model = Model(encoder_input, [combined_h, combined_c]) # This is the encoder inference model.


# Decoder Inference Model
decoded_input = Input(shape = (1, ))
decoder_input_h = Input(shape = (units*2, ))
decoder_input_c = Input(shape = (units*2, ))

decoder_emb2 = Embedding(fr_vocab, embedding_dim, mask_zero = True)(decoder_input)
decoder_outputs2, decoder_output_h, decoder_output_c =  decoder_lstm(decoder_emb2, initial_state = [decoder_input_h, decoder_input_c])
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model([decoder_input, decoder_input_h, decoder_input_c],
                      [decoder_outputs2, decoder_output_h, decoder_output_c])

# Translation:
def sentence_translation(sentence):
    sentence = ' '.join(smart_case([sentence], nlp_en))
    seq = eng_spm.EncodeAsIds(sentence)
    seq = pad_sequences([seq], maxlen = eng_max_len, padding = 'post') # Since we pass lists of tokenized sentences to pad_sequences(see the above eng_seq output).
    state_h, state_c = encoder_model.predict(seq) # Because encoder_model returns two states

    decoded_sentence = []
    start_id = fr_spm.PieceToId('<start>')
    input_id = np.array([[start_id]])
    
    stop = False

    while not stop:
        output_, output_h, output_c = decoder_model.predict([input_id, state_h, state_c])
        output_id = np.argmax(output_[0, -1, :])
        word = fr_spm.DecodeIds([output_id])

        if word == '<end>' or len(decoded_sentence) > fr_max_len:
            stop = True

        else:
            decoded_sentence.append(word)
            input_id = np.array([[output_id]])
            state_h, state_c = output_h, output_c

    return ' '.join(decoded_sentence)
    

In [ ]:
# ----------
# Testing-Phase
# ----------
sentences = ["I love eating apples.", "Please forgive me.", "Stay alert!"]
predictions = []

for s in sentences:
    predictions.append(sentence_translation(s))
    print(s)


# ----------------------
# Evaluation-Phase
# ----------------------
from nltk.translate.bleu_score import sentence_bleu
scores = []
references = ["J'adore manger des pommes.","Pardonnez-moi, s'il vous plaît.", "Restez vigilant(e) !"]

for ref, pred in zip(references, predictions):
    score = sentence_bleu([ref.split()], [pred.split()])
    scores.append(score)

print(f"Average Bleu Score: {sum(scores) / len(scores)}.:4")